In [1]:
%config IPCompleter.use_jedi = False
%load_ext autoreload
%autoreload 3
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
from threading import Thread

import h5py
import hdf5storage # conda install hdf5storage
import ipywidgets as widgets


# # required to enable non-blocking interaction:
%gui qt5

from pyphocorehelpers.Filesystem.HDF5.hdf5_file_helpers import h5dump, HDF5_Helper
from pyphocorehelpers.Filesystem.HDF5.silx_hdf5_viewer import hdf5_tree_view_window

from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, FileList
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_files_metadata
# from pyphocorehelpers.print_helpers import debug_dump_object_member_shapes, print_value_overview_only, pprint, debug_print, print_keys_if_possible
from pyphoplacecellanalysis.PhoPositionalData.load_exported import import_mat_file
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, extract_spike_timeseries
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import _test_save_pipeline_data_to_h5, get_h5_data_keys, save_some_pipeline_data_to_h5, load_pipeline_data_from_h5  #ExportHelpers

from pyphoplacecellanalysis.GUI.PyQtPlot.Params.MatFileBrowser.MatFileBrowser import MatFileBrowser

## Drastically improve Python Tracebacks on exceptions:
# from rich.traceback import install
# install()

In [ ]:
# RoyMaze1:

# mat_import_parent_path = Path(r'R:\data\RoyMaze1')
# mat_import_parent_path = Path(r'C:\Share\data\RoyMaze1') # Old one
# mat_import_file = r'C:\Share\data\RoyMaze1\ExportedData.mat'
mat_import_parent_path = Path(r'W:\\Data\\Kdiba\\**\\*.position_info.mat').resolve()

found_position_info_mat_files = MatFileBrowser.discover_mat_files(mat_import_parent_path, recursive=True)
found_position_info_mat_files

In [ ]:
search_parent_path = Path(r'W:\\Data\\Kdiba')
found_any_position_info_mat_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='*.position_info.mat', recursive=True)
found_any_position_info_mat_files


In [2]:
from pathlib import Path
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import batch_build_user_annotation_grid_bin_bounds_from_exported_position_info_mat_files

_out_user_annotations_add_code_lines, loaded_configs = batch_build_user_annotation_grid_bin_bounds_from_exported_position_info_mat_files(search_parent_path=Path(r'W:\\Data\\Kdiba'))
loaded_configs

file_prefix: 2006-6-07_11-26-53
session_name: 2006-6-07_11-26-53
_test_session_context: kdiba_gor01_one_2006-6-07_11-26-53
Loading matlab import file results : W:\Data\Kdiba\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat... done.
file_prefix: 2006-6-07_11-26-53
session_name: bak
file_prefix: 2006-6-07_11-26-53
session_name: essential
file_prefix: 2006-6-07_11-26-53
session_name: mat_test_BAK
file_prefix: 2006-6-08_14-26-15
session_name: 2006-6-08_14-26-15
_test_session_context: kdiba_gor01_one_2006-6-08_14-26-15
Loading matlab import file results : W:\Data\Kdiba\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.position_info.mat... done.
file_prefix: 2006-6-08_14-26-15
session_name: bak
file_prefix: 2006-6-09_1-22-43
session_name: 2006-6-09_1-22-43
_test_session_context: kdiba_gor01_one_2006-6-09_1-22-43
Loading matlab import file results : W:\Data\Kdiba\gor01\one\2006-6-09_1-22-43\2006-6-09_1-22-43.position_info.mat... done.
file_prefix: 2006-6-09_3-23-37
session_name

{WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-07_11-26-53/2006-6-07_11-26-53.position_info.mat'): {'position_sampling_rate_Hz': 29.96976599553983,
  'microseconds_to_seconds_conversion_factor': 1e-06,
  'pix2cm': 287.7697841726619,
  'x_midpoint': 143.88489208633095,
  'loaded_track_limits': {'long_xlim': array([ 59.07738974, 228.69004399]),
   'short_xlim': array([ 94.01556371, 193.7565709 ]),
   'long_ylim': array([135.23924312, 144.19845184]),
   'short_ylim': array([135.81263248, 144.62849386])},
  'session_context': Context(format_name: 'kdiba'
          animal: 'gor01'
          exper_name: 'one'
          session_name: '2006-6-07_11-26-53'),
  'new_grid_bin_bounds': [37.0773897438341,
   250.69004399129707,
   135.23924311831908,
   144.1984518410047]},
 WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-08_14-26-15/2006-6-08_14-26-15.position_info.mat'): {'position_sampling_rate_Hz': 29.96977250291495,
  'microseconds_to_seconds_conversion_factor': 1e-06,
  'pix2cm': 287.7697841726619,

In [ ]:
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder, DataSessionFormatBaseRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphocorehelpers.geometry_helpers import BoundsRect


## INPUTS: found_any_position_info_mat_files
platform_side_length: float = 22.0

user_annotations = UserAnnotationsManager.get_user_annotations()
_out_user_annotations_add_code_lines = []


loaded_configs = {}
for a_path in found_any_position_info_mat_files:
    if a_path.exists():
        file_prefix: str = a_path.name.split('.position_info.mat', maxsplit=1)[0]
        print(f'file_prefix: {file_prefix}')
        session_path = a_path.parent.resolve()
        session_name = KDibaOldDataSessionFormatRegisteredClass.get_session_name(session_path)
        print(f'session_name: {session_name}')
        if session_name == file_prefix:
            try:
                _test_session = KDibaOldDataSessionFormatRegisteredClass.build_session(session_path) # Path(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')
                _test_session_context = _test_session.get_context()
                print(f'_test_session_context: {_test_session_context}')
                
                _updated_config_dict = KDibaOldDataSessionFormatRegisteredClass.perform_load_position_info_mat(session_position_mat_file_path=a_path, config_dict={})
                _updated_config_dict['session_context'] = _test_session_context

                loaded_track_limits = _updated_config_dict.get('loaded_track_limits', None) # {'long_xlim': array([59.0774, 228.69]), 'short_xlim': array([94.0156, 193.757]), 'long_ylim': array([138.164, 146.12]), 'short_ylim': array([138.021, 146.263])}
                if loaded_track_limits is not None:
                    long_xlim = loaded_track_limits['long_xlim']
                    long_ylim = loaded_track_limits['long_ylim']
                    # short_xlim = loaded_track_limits['short_xlim']
                    # short_ylim = loaded_track_limits['short_ylim']

                    ## Build full grid_bin_bounds:
                    from_mat_lims_grid_bin_bounds = BoundsRect(xmin=(long_xlim[0]-platform_side_length), xmax=(long_xlim[1]+platform_side_length), ymin=long_ylim[0], ymax=long_ylim[1])
                    # print(f'from_mat_lims_grid_bin_bounds: {from_mat_lims_grid_bin_bounds}')
                    # from_mat_lims_grid_bin_bounds # BoundsRect(xmin=37.0773897438341, xmax=250.69004399129707, ymin=116.16397564990257, ymax=168.1197529956474)
                    _updated_config_dict['new_grid_bin_bounds'] = from_mat_lims_grid_bin_bounds.extents

                    ## Build Update:


                    # curr_active_pipeline.sess.config.computation_config['pf_params'].grid_bin_bounds = new_grid_bin_bounds

                    active_context = _test_session_context # curr_active_pipeline.get_session_context()
                    final_context = active_context.adding_context_if_missing(user_annotation='grid_bin_bounds')
                    user_annotations[final_context] = from_mat_lims_grid_bin_bounds.extents
                    # Updates the context. Needs to generate the code.

                    ## Generate code to insert int user_annotations:
                    
                    ## new style:
                    # print(f"user_annotations[{final_context.get_initialization_code_string()}] = {(from_mat_lims_grid_bin_bounds.xmin, from_mat_lims_grid_bin_bounds.xmax), (from_mat_lims_grid_bin_bounds.ymin, from_mat_lims_grid_bin_bounds.ymax)}")
                    _a_code_line: str = f"user_annotations[{active_context.get_initialization_code_string()}] = dict(grid_bin_bounds=({(from_mat_lims_grid_bin_bounds.xmin, from_mat_lims_grid_bin_bounds.xmax), (from_mat_lims_grid_bin_bounds.ymin, from_mat_lims_grid_bin_bounds.ymax)}))"
                    # print(_a_code_line)
                    _out_user_annotations_add_code_lines.append(_a_code_line)

                loaded_configs[a_path] = _updated_config_dict
            except (AssertionError, BaseException) as e:
                print(f'e: {e}. Skipping.')
                pass
                
# loaded_configs

print('Add the following code to UserAnnotationsManager.get_user_annotations() function body:')
print('\n'.join(_out_user_annotations_add_code_lines))

user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-07_11-26-53')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (135.23924311831908, 144.1984518410047))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (137.925447118083, 145.16448776601297))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (138.16397564990257, 146.1197529956474))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_3-23-37')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (139.34507862499134, 147.58755064986212))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (137.97626338793503, 146.00371440346137))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (138.2791990765373, 145.4465660546858))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (138.39266723911976, 146.9470603477007))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_15-46-47')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (142.39397650284766, 151.15367504603452))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (139.89578722770338, 148.51861548115295))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (139.3053644252862, 147.99662782360443))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (139.91706986262813, 148.7452035506836))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (141.07167669307665, 149.9682455260008))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (136.10095530133273, 144.59252270314897))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (141.44749773951332, 147.73754766678462))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (138.9314777686048, 146.63678892951214))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_21-26-8')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (138.0941872269056, 144.73231230966147))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-05_19-26-43')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (142.68438010327935, 149.1812684821468))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_11-43-50')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (141.83695987994884, 148.19261155492785))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (143.81427373438675, 151.01734563269633))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_21-17-16')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (143.1080902149446, 149.67559694575627))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (141.6251048241162, 147.8395197952068))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (140.14211943328775, 147.2745729796531))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_20-28-3')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (137.74109546718452, 144.66169395771726))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-04_21-20-3')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (138.05172078553375, 143.70812931964014))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (108.13933162527657, 115.13913718623321))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (107.8177789584226, 113.7570079192343))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_21-2-40')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (108.18519657066037, 113.91231021144307))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-11_15-16-59')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (108.52716865032582, 113.61793633102158))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_14-39-31')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (109.19374317979644, 114.35521596716852))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_17-53-55')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (107.82954763216065, 113.6273663796197))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-16_15-12-23')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (108.31665318697634, 113.76094640105372))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (109.64675309404181, 115.02034120144286))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (108.9445597322445, 114.6716733730272))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_15-23-32')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (109.67092311452707, 114.54957547519382))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (106.80073123839011, 112.7399601992018))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_16-48-9')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (108.55947898953707, 114.56941305702512))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-21_10-24-35')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (108.22908621020602, 114.94607134445735))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_14-28-51')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (109.0566791531139, 115.63225407401256))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_17-17-6')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (109.29954776242832, 115.16807161656368))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-26_13-22-13')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (107.60152468408094, 114.81344556506657))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (107.22404579212474, 114.08244113972872))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_16-48-29')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (108.2501053175898, 113.9772189583725))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (122.96634289724359, 130.46108420493456))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (122.84017431563773, 129.69856966324173))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_19-11-57')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (122.9536718458597, 129.52924676675838))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (125.03673561741104, 131.04666968489906))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.05348636842058, 129.56848468917428))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (126.17725557343053, 131.1266130407736))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (123.90636700893516, 129.5627755430415))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (123.63448954400148, 128.79596233137354))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.14624025653195, 129.44912325725667))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_13-28-57')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (122.92990621750084, 128.7984300716362))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_15-38-2')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.22934368316666, 129.46152157721505))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.50144550147462, 129.4508029688177))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_16-37-40')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.27871634925401, 129.58159934997875))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-21_11-19-2')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.01748457963947, 129.67389311374583))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.59625706838818, 130.18196049581826))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_17-33-28')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.54109241108247, 129.91468051848352))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (123.37674715861888, 129.74020675948856))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (124.79328080534583, 130.59109955280485))))
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_17-6-14')] = dict(grid_bin_bounds=(((37.0773897438341, 250.69004399129707), (123.16323999738597, 128.89035363816865))))

In [ ]:
found_file_variables_dict = MatFileBrowser.whos_mat_files(found_any_position_info_mat_files)
tree, app = MatFileBrowser.build_browsing_gui(found_file_variables_dict)
tree.show()


In [ ]:
search_parent_path = Path(r'W:\\Data\\Kdiba')
found_any_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='*.pkl', recursive=True) # any pickle, anywhere

found_any_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='*.pkl', recursive=True) # any pickle, anywhere
found_other_outputs_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='output/*.pkl', recursive=True) # any pickle in the `output/` folder


## Secondary types:
found_autoversioned_session_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='*-loadedSessPickle.pkl', recursive=True)


## Core desired types:
found_default_session_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='loadedSessPickle.pkl', recursive=True)
found_global_computation_results_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='output/global_computation_results.pkl', recursive=True) # 'output/global_computation_results.pkl'

all_core_desired_files = found_default_session_pickle_files + found_global_computation_results_files

# found_files: list[Path] = found_default_session_pickle_files + found_global_computation_results_files + found_autoversioned_session_pickle_files
# found_files

# non_primary_desired_files = to_set(found_any_pickle_files)[0] - to_set(all_core_desired_files)[0]
# non_primary_desired_files = from_set(to_set(found_any_pickle_files)[0] - to_set(found_default_session_pickle_files + found_global_computation_results_files)[0])
# non_primary_desired_files = FileList.from_set(FileList.to_set(found_any_pickle_files) - FileList.to_set(found_default_session_pickle_files + found_global_computation_results_files))
non_primary_desired_files = FileList.subtract(found_any_pickle_files, all_core_desired_files)
non_primary_desired_files

# all_core_desired_files
non_primary_desired_files


In [ ]:

# 'ripple_df.pkl', .ripple.npy'


session_folder_output_file_suffix_list = ['.interpolated_spike_positions.npy', ".mua.npy", '.pbe.npy', '.position.npy']


session.filePrefix.with_suffix('.interpolated_spike_positions.npy')
session.filePrefix.with_suffix(".mua.npy")


"""
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.interpolated_spike_positions.npy,"2023-06-07 01:03:36",35290406
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.mua.npy,"2023-06-07 01:03:37",7766554
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.pbe.npy,"2023-06-07 01:03:37",10902
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.position.npy,"2023-06-07 01:03:36",2446247
"""

- https://realpython.com/inherit-python-list/
- https://stackoverflow.com/questions/8180014/how-to-subclass-python-list-without-type-problems/8180073#8180073
- https://docs.python.org/3/reference/datamodel.html#emulating-container-types
- https://docs.python.org/3/library/collections.html#collections.UserList
- https://stackoverflow.com/questions/50470557/pythonic-way-of-subclassing-a-list
- https://vetude.com/part-9-list-in-python-an-introduction-to-subclassing-a-list/
- https://datagy.io/python-collections-userlist/
- https://sentry.io/answers/what-are-Python-metaclasses/
- https://stackoverflow.com/questions/25328448/should-i-subclass-list-or-create-class-with-list-as-attribute- 

In [ ]:
split_on_nearest_data_directory = lambda a_path: next(((tuple(a_path.relative_to(parent).parts[:-1]), tuple(a_path.relative_to(parent).parts[-1:])) for parent in a_path.parents if parent.name.lower() == 'data'), None)

# my_list = ['path', 'to', 'some', 'Data', 'directory', 'file.txt']
my_list = a_path.parts # ('W:\\', 'Data', 'Kdiba', 'gor01', 'one', '2006-6-07_11-26-53', 'loadedSessPickle.pkl')
search_element = 'Data'

# matches = [(i, elem) for i, elem in enumerate(reversed(my_list)) if elem.lower() == search_element.lower()]

try:
    index = len(my_list) - my_list[::-1].index('Data') - 1
    print("Index of the first element matching 'Data':", index)
    list_before = my_list[:index]
    list_after = my_list[index:]
    print("List before index:", list_before)
    print("List after index:", list_after)
    data_folder_relative_path = Path(*list_after)
    print(f'data_folder_relative_path: {data_folder_relative_path}')
except ValueError:
    print("No element matching 'Data' found in the list.")
    
a_path = all_core_desired_files[0]
(str(a_path.parent.as_posix()), data_folder_relative_path, a_path.name)
# a_path.as_posix()


# split_parts = split_on_nearest_data_directory(a_path)
# if split_parts:
#     before_data, after_data = split_parts
#     print('Before Data:', before_data)
#     print('After Data:', after_data)
# else:
#     print("No 'Data' or 'data' parent directory found in the path.")
    
# a_path.parts.reversed().index('Data')
# print(a_path.parts) # ('W:\\', 'Data', 'Kdiba', 'gor01', 'one', '2006-6-07_11-26-53', 'loadedSessPickle.pkl')
# index

In [ ]:
# pd.DataFrame({'path': found_files})

all_core_desired_files_df = get_files_metadata(all_core_desired_files)
all_core_desired_files_df['path']


all_core_desired_files_df['filename'] = 

In [ ]:
found_file_variables_dict = MatFileBrowser.whos_mat_files(found_files)
print(found_file_variables_dict)

# Silx hdf5 Widgets

In [ ]:
non_hdf5_files = [Path(r'R:/data/RoyMaze1/ExportedData/extrasAnalysis.mat'), Path('R:/data/RoyMaze1/ExportedData/RippleIndexedSpikes.mat')] # , Path(r'R:/data/RoyMaze1/ExportedData/positionAnalysis.mat')
hdf5_found_files = [str(found_file) for found_file in found_files if found_file not in non_hdf5_files]
print(f'hdf5_found_files: {hdf5_found_files}')
win = hdf5_tree_view_window(hdf5_found_files)
win

# Overflow (Not needed, Slix HDF5 widget worked to display/browse the data)

In [ ]:
# Import the positions
t,x,y,speeds,dt,dx,dy = perform_import_positions(mat_import_parent_path=mat_import_parent_path)
# Import the spikes
spike_matrix, spike_cells, num_cells, spike_list, spike_positions_list = perform_import_spikes(t, x, y, mat_import_parent_path=mat_import_parent_path)

ripple_times, ripple_peak_times, ripple_peak_values, behavioral_periods, behavioral_epochs = perform_import_extras(mat_import_parent_path=mat_import_parent_path)

# behavioral_periods = all_results_data['active_processing/behavioral_periods_table']
# print('spike_matrix: {}, spike_cells: {}'.format(np.shape(spike_matrix), np.shape(spike_cells)))
num_periods = np.shape(behavioral_periods)[0]
print('num_periods: {}'.format(num_periods))


# creating the dataset
#date = [i for i in range(2000, 2021)]
 
# Create a figure and a set of subplots
fig, ax = plt.subplots(figsize=(10, 6))
 
# Creating the scatter plot
#ax.scatter(date, value)
 
# Highlighting for a certain period of time
num_ripples = np.shape(ripple_times)[0]
for i in np.arange(num_ripples):
    ax.axvspan(ripple_times[i,0], ripple_times[i,1], alpha=0.3, color="green")

plt.show()

In [ ]:
import silx.gui.hdf5
treeview = silx.gui.hdf5.Hdf5TreeView()
model = treeview.findHdf5TreeModel()
# Avoid the user to drop file in the widget
model.setFileDropEnabled(False)
# Allow the user to reorder files with drag-and-drop
model.setFileMoveEnabled(True)
header = treeview.header()
# Select displayed columns
column_ids = [treeview.findHdf5TreeModel().NAME_COLUMN]
header.setSections(column_ids)
# Do not allow the user to custom visible columns
header.setEnableHideColumnsPopup(False)

## MatFileBrowser.build_browsing_gui(...) Parameter Tree GUI (WORKING, NOT USED)

In [ ]:
tree, app = MatFileBrowser.build_browsing_gui(found_file_variables_dict)
tree.show()

## HDF5_Helper Recurrsion Testing (NOT FINISHED, NOT WORKING, NOT USED)

In [ ]:
# found_file = Path(r'R:/data/RoyMaze1/analysesResults_30-Sep-2021/allEnv.mat')
# found_file = Path(r'R:/data/RoyMaze1/ExportedData/spikesAnalysis.mat')
# found_file = Path(r'R:\data\RoyMaze1\ExportedData\extrasAnalysis.mat')
found_file = Path(r'R:/data/RoyMaze1/ExportedData/spikesTable.mat')

In [ ]:
# import h5py
helper = HDF5_Helper(path=found_file, group='active_processing')
helper.output_tree

In [ ]:
# helper.perform_enumerate(debug_print=True)
helper.perform_enumerate(debug_print=False)
helper.output_tree

In [ ]:
found_file_variables_dict[found_file]

In [ ]:
h5dump(found_file, group='active_processing', enable_print_attributes=False)

## h5py 'visit_item' testing (WORKING, NOT USED):

In [ ]:
def pho_visit_item_fn(name, obj):
    # Recursively visit all objects in this group and subgroups. Like Group.visit(), except your callable should have the signature:
    # callable(name, object) -> None or return value
    # In this case object will be a Group or Dataset instance.
    # if name not in ignored_keys:        
    if '#' in name:
        return None
    else:
        print(f'name: {name}, obj: {obj}')
        # if type(obj) in [h5py._hl.group.Group, h5py._hl.files.File]:
        #     for key in obj.keys():
        #         print(sep,'-',key,':',obj[key])
        #         descend_obj(obj[key],sep=sep+'\t')
        # elif type(obj)==h5py._hl.dataset.Dataset:
        #     for key in obj.attrs.keys():
        #         print(sep+'\t','-',key,':',obj.attrs[key])
            
    
def find_foo(name):
    """ Find first object with 'foo' anywhere in the name """
    if 'foo' in name:
        return name

with h5py.File(found_file,'r') as f:
    print(f'file:{found_file}: {f}')
    # print(f.visititems)
    _out = f.visititems(pho_visit_item_fn)
    # group.visit(find_foo)

    # f.attrs
    # print_value_overview_only(f)
    # # found_data_dict = {a_key:meta_info_dict for a_key in list(f.keys()) if a_key not in ignored_keys} ## filter the ignored keys
    # # found_keys = [a_key for a_key in list(f.keys()) if a_key not in ignored_keys] ## filter the ignored keys
    # f.name

## QFileSystemModel, QTreeView: Fresh Implementation (NOT USED)

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QFileSystemModel, QTreeView, QWidget, QVBoxLayout
from PyQt5.QtGui import QIcon

class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'PyQt5 file system view - pythonspot.com'
        self.left = 10
        self.top = 10
        self.width = 640
        self.height = 480
        self.initUI()
    
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        self.model = QFileSystemModel()
        self.model.setRootPath(r'E:\data')
        self.tree = QTreeView()
        self.tree.setModel(self.model)
        
        self.tree.setAnimated(False)
        self.tree.setIndentation(20)
        self.tree.setSortingEnabled(True)
        
        self.tree.setWindowTitle("Dir View")
        self.tree.resize(640, 480)
        
        windowLayout = QVBoxLayout()
        windowLayout.addWidget(self.tree)
        self.setLayout(windowLayout)
        
        self.show()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())